ICESat-2 orientation can be: backward, forward, and transition. During transition, ICESat-2 is maneuvering between the two orientations. Science quality is potentially degraded while in transition mode. (Source: https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL03_data_dict_v005.pdf)


The parameter sc_orient in ATL03 denotes the orientation of ICESat-2
0 = backward
1 = forward
2 = transition